In [ ]:
#下載台北市youbike資料
import requests
response = requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
response.encoding = 'utf-8'
youbikeData = response.json()
youbikeData

In [3]:
import psycopg2
import os
conn = psycopg2.connect(host="localhost",database='postgres',user=os.environ["POSTGRES_USER"],password=os.environ["POSTGRES_PASSWORD"])

with conn:
    with conn.cursor() as cursor:
        #insert站點資訊
        for site in youbikeData:
            sql = '''
                INSERT INTO 站點資訊 
                VALUES(%s,%s,%s,%s,%s,%s)
                ON CONFLICT DO NOTHING;
            '''
            cursor.execute(sql,(site['sno'],site['sna'],site['sarea'],site['ar'],site['lat'],site['lng']))

            #插入youbike資訊
            sql = '''
                INSERT INTO youbike
                VALUES(%s,%s,%s,%s,%s,%s)
                ON CONFLICT DO NOTHING;
            '''
            cursor.execute(sql,(site['mday'],site['sno'],site['tot'],site['sbi'],site['bemp'],bool(site['act'])))


conn.close()